In [ ]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

In [106]:
import pandas as pd
import numpy as np
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC

from sklearn.model_selection import StratifiedKFold 

model = LinearSVC(dual=False,tol=1e-3, class_weight = 'balanced') #please try with different versions of gamma and 'C' parameters here
                  
df = pd.read_csv('telekomexportsichter11.csv')
df1 = pd.read_csv('telekomexportsichter12.csv')
df2 = pd.read_csv('telekomexportsichter13.csv')
df3 = pd.read_csv('telekomexportsichter14.csv')
df4 = pd.read_csv('telekomexportsichter15.csv')
df5 = pd.read_csv('telekomexportsichter16.csv')
df6 = pd.read_csv('telekomexportsichter17.csv')
df7 = pd.read_csv('telekomexportsichter18.csv')

df = df.append(df1)
df = df.append(df2)
df = df.append(df3)
df = df.append(df4)
df = df.append(df5)
df = df.append(df6)
df = df.append(df7)


priorCases = df[(df['Tag'] == 'Prio-Fall') & (df['kind'] == 1)]['Content']
priorCases = pd.Series.to_frame(priorCases)

nonPriorCases = df[(df['Tag'] != 'Prio-Fall') & (df['kind'] == 1)]['Content']
nonPriorCases = pd.Series.to_frame(nonPriorCases)

priorCases = priorCases.assign(label = 1)
nonPriorCases = nonPriorCases.assign(label = 0)
#priorCases.drop(priorCases.index[priorCases['Content'].isnull()])
#priorCases[priorCases['Content'].isnull()].head
priorCases = priorCases.append(nonPriorCases)

In [110]:
priorCases = priorCases.drop(priorCases.index[priorCases['Content'].isnull()])
cv = CountVectorizer(min_df = 3, max_df = 10000, ngram_range=(1, 3), lowercase =True) #change the ngram range to 5,7, maximum 9 

In [111]:
print(priorCases.index[priorCases['Content'].isnull()])

Int64Index([], dtype='int64')


In [113]:
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=0) #do not use more or less splits
scores = []
for train_index, test_index in skf.split(priorCases['Content'] , priorCases['label']):
    #priorCases.loc[train_index]['Content']
    
    model = LinearSVC(dual=False,tol=1e-3, class_weight = 'balanced')
    priorCases2 = priorCases.loc[train_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    X_train_counts = cv.fit_transform(priorCases2['Content'])
    #print(X_train_counts.shape)


    tfidfTransform = TfidfTransformer()
    X_Train_tfidf = tfidfTransform.fit_transform(X_train_counts)
    #print(X_Train_tfidf.shape)
    
    priorCases2 = priorCases.loc[test_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    X_test_counts = cv.transform(priorCases2['Content'])
    #print(X_test_counts.shape)

    X_test_tfidf = tfidfTransform.transform(X_test_counts)
    #print(X_test_tfidf.shape)
    
    priorCases2 = priorCases.loc[train_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    model.fit(X_Train_tfidf, priorCases2['label'])
    
    pred = model.predict(X_test_tfidf)
    
    priorCases2 = priorCases.loc[test_index]
    priorCases2 = priorCases2.drop(priorCases2.index[priorCases2['Content'].isnull()])
    answer = sklearn.metrics.f1_score(priorCases2['label'], pred)
    #print(sklearn.metrics.confusion_matrix(priorCases2['label'], pred, labels=None, sample_weight=None))
    print(np.mean(pred == priorCases2['label']))
    print("F1")
    print(answer)

0.8988903394255875
F1
0.5304637769020915
0.9009106394129979
F1
0.521436481569372
0.896859675900367
F1
0.5182038834951457
0.8994355075537573
F1
0.5391746411483254
0.8999054481431972
F1
0.5241785492870428


In [ ]:
X_train_counts = cv.fit_transform(labeled['Content'])
X_Train_tfidf = tfidfTransform.fit_transform(X_train_counts)

X_test_counts = cv.transform(notLabeled['Content'])
X_test_tfidf = tfidfTransform.transform(X_test_counts)

model = LinearSVC(dual=False,tol=1e-3, class_weight = 'balanced')
model.fit(X_Train_tfidf, priorCases2['SichterName'])